In [1]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

import tensorflow_datasets as tfds
# Tensorflow가 활용할 GPU가 장착되어 있는지 확인해 봅니다.
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
# function for building ResNet Block

def build_resnet_block34(input_layer,
                       num_cnn=3,
                       channel_ls=[64, 64],
                       kernel_size_ls=[3, 3],
                       block_num=2,
                      ):
    # 입력 레이어
    inputs = input_layer
    x = inputs

    # CNN 레이어
    for cnn_num in range(num_cnn):
        for idx, (channel, kernel_size) in enumerate(zip(channel_ls, kernel_size_ls)):
            x = keras.layers.Conv2D(
                filters=channel,
                kernel_size=(kernel_size, kernel_size),
                kernel_initializer='he_normal',
                padding='same',
                name=f'block{block_num}_conv{cnn_num+1}_{idx+1}'
            )(x)
            x = keras.layers.BatchNormalization()(x)
            
            if idx+1 >= len(channel_ls):
                break
            
            x = keras.layers.Activation('relu')(x)
                
        x = keras.layers.Add()([x, inputs])
        x = keras.layers.Activation('relu')(x)

    return x

In [3]:
# function for building ResNet Block

def build_resnet_block50(input_layer,
                       num_cnn=3,
                       channel_ls=[64, 64, 256],
                       kernel_size_ls=[3, 3],
                       block_num=2,
                      ):
    # 입력 레이어
    inputs = input_layer
    inputs_conv = keras.layers.Conv2D(
                                filters=channel_ls[-1],  # 2배로 증가
                                kernel_size=(1, 1),       # 1×1 컨볼루션
                                strides=1,
                                padding='same',           # 패딩 유지
                                kernel_initializer='he_normal'  # He 초기화
                            )(inputs)
    x = inputs

    # CNN 레이어
    for cnn_num in range(num_cnn):
        for idx, (channel, kernel_size) in enumerate(zip(channel_ls, kernel_size_ls)):
            x = keras.layers.Conv2D(
                filters=channel,
                kernel_size=(kernel_size, kernel_size),
                kernel_initializer='he_normal',
                padding='same',
                name=f'block{block_num}_conv{cnn_num+1}_{idx+1}'
            )(x)
            x = keras.layers.BatchNormalization()(x)
            
            if idx+1 >= len(channel_ls):
                break
            
            x = keras.layers.Activation('relu')(x)
        
        x = keras.layers.Add()([x, inputs_conv])
        x = keras.layers.Activation('relu')(x)

    return x

In [4]:
resnet_input_layer = keras.layers.Input(shape=(56,56,64))
resnet_block_output = build_resnet_block34(input_layer=resnet_input_layer,
                       num_cnn=3,
                       channel_ls=[64, 64],
                       kernel_size_ls=[3, 3],
                       block_num=2,
                      )

model = keras.Model(inputs=resnet_input_layer, outputs=resnet_block_output)  
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 56, 56, 64)] 0                                            
__________________________________________________________________________________________________
block2_conv1_1 (Conv2D)         (None, 56, 56, 64)   36928       input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 56, 56, 64)   256         block2_conv1_1[0][0]             
__________________________________________________________________________________________________
activation (Activation)         (None, 56, 56, 64)   0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [5]:
del model

In [6]:
# ResNet모델 자체를 생성하는 함수입니다.
def build_resnet34(input_shape=(224,224,3),
              num_cnn_list=[3,4,6,3],
              channel_list=[64,128,256,512],
              num_classes=10):
    
    assert len(num_cnn_list) == len(channel_list) #모델을 만들기 전에 config list들이 같은 길이인지 확인합니다.
    
    input_layer = keras.layers.Input(shape=input_shape)  # input layer를 만들어둡니다.
    
    output = keras.layers.Conv2D(
                filters=64,
                kernel_size=(7, 7),
                strides=2,
                padding='same',
                name=f'block1_conv'
            )(input_layer)
    
    output = keras.layers.BatchNormalization()(output)
    output = keras.layers.Activation('relu')(output)
    
    output = keras.layers.MaxPooling2D(
                pool_size=(3, 3),
                strides=2,
                padding='same',
                name=f'block1_maxpool'
            )(output)
    
    # config list들의 길이만큼 반복해서 블록을 생성합니다.
    block_num=2
    for i, (num_cnn, channel) in enumerate(zip(num_cnn_list, channel_list)):
        output = build_resnet_block34(input_layer=output,
                       num_cnn=num_cnn,
                       channel_ls=[channel, channel],
                       kernel_size_ls=[3, 3],
                       block_num=block_num,
                      )
        
        if i+1 >= len(num_cnn_list):
            break
        
        output = keras.layers.Conv2D(
                                filters=channel*2,  # 2배로 증가
                                kernel_size=(1, 1),       # 1×1 컨볼루션
                                strides=1,
                                padding='same',           # 패딩 유지
                                kernel_initializer='he_normal'  # He 초기화
                            )(output)
        block_num+=1
        
    output = keras.layers.AveragePooling2D(name='AveragePooling2D')(output)
    output = keras.layers.Flatten()(output)
    output = keras.layers.Dense(num_classes, activation='softmax', name='predictions')(output)
    
    model = keras.Model(
        inputs=input_layer, 
        outputs=output
    )
    return model

In [7]:
# ResNet모델 자체를 생성하는 함수입니다.
def build_resnet50(input_shape=(224,224,3),
              num_cnn_list=[3,4,6,3],
              channel_list=[64,128,256,512],
              num_classes=10):
    
    assert len(num_cnn_list) == len(channel_list) #모델을 만들기 전에 config list들이 같은 길이인지 확인합니다.
    
    input_layer = keras.layers.Input(shape=input_shape)  # input layer를 만들어둡니다.
    
    output = keras.layers.Conv2D(
                filters=64,
                kernel_size=(7, 7),
                strides=2,
                padding='same',
                name=f'block1_conv'
            )(input_layer)
    
    output = keras.layers.BatchNormalization()(output)
    output = keras.layers.Activation('relu')(output)
    
    output = keras.layers.MaxPooling2D(
                pool_size=(3, 3),
                strides=2,
                padding='same',
                name=f'block1_maxpool'
            )(output)
    
    # config list들의 길이만큼 반복해서 블록을 생성합니다.
    block_num=2
    for i, (num_cnn, channel) in enumerate(zip(num_cnn_list, channel_list)):
        output = build_resnet_block50(input_layer=output,
                       num_cnn=num_cnn,
                       channel_ls=[channel, channel, channel*4],
                       kernel_size_ls=[1, 3, 1],
                       block_num=block_num,
                      )
        
        if i+1 >= len(num_cnn_list):
            break
        
        output = keras.layers.Conv2D(
                                filters=channel*2,  # 2배로 증가
                                kernel_size=(1, 1),       # 1×1 컨볼루션
                                strides=1,
                                padding='same',           # 패딩 유지
                                kernel_initializer='he_normal'  # He 초기화
                            )(output)
        block_num+=1
        
    output = keras.layers.AveragePooling2D(name='AveragePooling2D')(output)
    output = keras.layers.Flatten()(output)
    output = keras.layers.Dense(num_classes, activation='softmax', name='predictions')(output)
    
    model = keras.Model(
        inputs=input_layer, 
        outputs=output
    )
    return model

In [8]:
def build_resnet(input_shape=(224,224,3),
              num_cnn_list=[3,4,6,3],
              channel_list=[64,128,256,512],
              num_classes=10,
                is_50=False):
    if not is_50:
        return build_resnet34(input_shape=input_shape,
              num_classes=num_classes)
    
    else:
        return build_resnet50(input_shape=input_shape,
              num_classes=num_classes)
        

In [9]:
# 원하는 블록의 설계에 따라 매개변수로 리스트를 전달해 줍니다.
resnet_34 = build_resnet(input_shape=(32,32,3))

resnet_34.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
block1_conv (Conv2D)            (None, 16, 16, 64)   9472        input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_6 (BatchNor (None, 16, 16, 64)   256         block1_conv[0][0]                
__________________________________________________________________________________________________
activation_6 (Activation)       (None, 16, 16, 64)   0           batch_normalization_6[0][0]      
____________________________________________________________________________________________

In [10]:
del resnet_34

In [11]:
# 원하는 블록의 설계에 따라 매개변수로 리스트를 전달해 줍니다.
resnet_50 = build_resnet(input_shape=(32,32,3), is_50=True)

resnet_50.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
block1_conv (Conv2D)            (None, 16, 16, 64)   9472        input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_39 (BatchNo (None, 16, 16, 64)   256         block1_conv[0][0]                
__________________________________________________________________________________________________
activation_39 (Activation)      (None, 16, 16, 64)   0           batch_normalization_39[0][0]     
____________________________________________________________________________________________

In [12]:
del resnet_50

In [13]:
# function for building ResNet Block

def build_plainnet_block34(input_layer,
                       num_cnn=3,
                       channel_ls=[64, 64],
                       kernel_size_ls=[3, 3],
                       block_num=2,
                      ):
    # 입력 레이어
    inputs = input_layer
    x = inputs

    # CNN 레이어
    for cnn_num in range(num_cnn):
        for idx, (channel, kernel_size) in enumerate(zip(channel_ls, kernel_size_ls)):
            x = keras.layers.Conv2D(
                filters=channel,
                kernel_size=(kernel_size, kernel_size),
                kernel_initializer='he_normal',
                padding='same',
                name=f'block{block_num}_conv{cnn_num+1}_{idx+1}'
            )(x)
            x = keras.layers.BatchNormalization()(x)
            
            if idx+1 >= len(channel_ls):
                break
            
            x = keras.layers.Activation('relu')(x)
                
#         x = keras.layers.Add()([x, inputs])
        x = keras.layers.Activation('relu')(x)

    return x

In [14]:
# function for building ResNet Block

def build_plainnet_block50(input_layer,
                       num_cnn=3,
                       channel_ls=[64, 64, 256],
                       kernel_size_ls=[3, 3],
                       block_num=2,
                      ):
    # 입력 레이어
    inputs = input_layer
#     inputs_conv = keras.layers.Conv2D(
#                                 filters=channel_ls[-1],  # 2배로 증가
#                                 kernel_size=(1, 1),       # 1×1 컨볼루션
#                                 strides=1,
#                                 padding='same',           # 패딩 유지
#                                 kernel_initializer='he_normal'  # He 초기화
#                             )(inputs)
    x = inputs

    # CNN 레이어
    for cnn_num in range(num_cnn):
        for idx, (channel, kernel_size) in enumerate(zip(channel_ls, kernel_size_ls)):
            x = keras.layers.Conv2D(
                filters=channel,
                kernel_size=(kernel_size, kernel_size),
                kernel_initializer='he_normal',
                padding='same',
                name=f'block{block_num}_conv{cnn_num+1}_{idx+1}'
            )(x)
            x = keras.layers.BatchNormalization()(x)
            
            if idx+1 >= len(channel_ls):
                break
            
            x = keras.layers.Activation('relu')(x)
        
#         x = keras.layers.Add()([x, inputs_conv])
        x = keras.layers.Activation('relu')(x)

    return x

In [15]:
# ResNet모델 자체를 생성하는 함수입니다.
def build_plainnet34(input_shape=(224,224,3),
              num_cnn_list=[3,4,6,3],
              channel_list=[64,128,256,512],
              num_classes=10):
    
    assert len(num_cnn_list) == len(channel_list) #모델을 만들기 전에 config list들이 같은 길이인지 확인합니다.
    
    input_layer = keras.layers.Input(shape=input_shape)  # input layer를 만들어둡니다.
    
    output = keras.layers.Conv2D(
                filters=64,
                kernel_size=(7, 7),
                strides=2,
                padding='same',
                name=f'block1_conv'
            )(input_layer)
    
    output = keras.layers.BatchNormalization()(output)
    output = keras.layers.Activation('relu')(output)
    
    output = keras.layers.MaxPooling2D(
                pool_size=(3, 3),
                strides=2,
                padding='same',
                name=f'block1_maxpool'
            )(output)
    
    # config list들의 길이만큼 반복해서 블록을 생성합니다.
    block_num=2
    for i, (num_cnn, channel) in enumerate(zip(num_cnn_list, channel_list)):
        output = build_plainnet_block34(input_layer=output,
                       num_cnn=num_cnn,
                       channel_ls=[channel, channel],
                       kernel_size_ls=[3, 3],
                       block_num=block_num,
                      )
        
        if i+1 >= len(num_cnn_list):
            break
        
        output = keras.layers.Conv2D(
                                filters=channel*2,  # 2배로 증가
                                kernel_size=(1, 1),       # 1×1 컨볼루션
                                strides=1,
                                padding='same',           # 패딩 유지
                                kernel_initializer='he_normal'  # He 초기화
                            )(output)
        block_num+=1
        
    output = keras.layers.AveragePooling2D(name='AveragePooling2D')(output)
    output = keras.layers.Flatten()(output)
    output = keras.layers.Dense(num_classes, activation='softmax', name='predictions')(output)
    
    model = keras.Model(
        inputs=input_layer, 
        outputs=output
    )
    return model

In [16]:
# ResNet모델 자체를 생성하는 함수입니다.
def build_plainnet50(input_shape=(224,224,3),
              num_cnn_list=[3,4,6,3],
              channel_list=[64,128,256,512],
              num_classes=10):
    
    assert len(num_cnn_list) == len(channel_list) #모델을 만들기 전에 config list들이 같은 길이인지 확인합니다.
    
    input_layer = keras.layers.Input(shape=input_shape)  # input layer를 만들어둡니다.
    
    output = keras.layers.Conv2D(
                filters=64,
                kernel_size=(7, 7),
                strides=2,
                padding='same',
                name=f'block1_conv'
            )(input_layer)
    
    output = keras.layers.BatchNormalization()(output)
    output = keras.layers.Activation('relu')(output)
    
    output = keras.layers.MaxPooling2D(
                pool_size=(3, 3),
                strides=2,
                padding='same',
                name=f'block1_maxpool'
            )(output)
    
    # config list들의 길이만큼 반복해서 블록을 생성합니다.
    block_num=2
    for i, (num_cnn, channel) in enumerate(zip(num_cnn_list, channel_list)):
        output = build_plainnet_block50(input_layer=output,
                       num_cnn=num_cnn,
                       channel_ls=[channel, channel, channel*4],
                       kernel_size_ls=[1, 3, 1],
                       block_num=block_num,
                      )
        
        if i+1 >= len(num_cnn_list):
            break
        
        output = keras.layers.Conv2D(
                                filters=channel*2,  # 2배로 증가
                                kernel_size=(1, 1),       # 1×1 컨볼루션
                                strides=1,
                                padding='same',           # 패딩 유지
                                kernel_initializer='he_normal'  # He 초기화
                            )(output)
        block_num+=1
        
    output = keras.layers.AveragePooling2D(name='AveragePooling2D')(output)
    output = keras.layers.Flatten()(output)
    output = keras.layers.Dense(num_classes, activation='softmax', name='predictions')(output)
    
    model = keras.Model(
        inputs=input_layer, 
        outputs=output
    )
    return model

In [17]:
def build_plainnet(input_shape=(224,224,3),
              num_cnn_list=[3,4,6,3],
              channel_list=[64,128,256,512],
              num_classes=10,
                is_50=False):

    if not is_50:
        return build_plainnet34(input_shape=input_shape,
              num_classes=num_classes)
    
    else:
        return build_plainnet50(input_shape=input_shape,
              num_classes=num_classes)
        

In [18]:
# 원하는 블록의 설계에 따라 매개변수로 리스트를 전달해 줍니다.
plaainnet_34 = build_plainnet(input_shape=(32,32,3))

plaainnet_34.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv (Conv2D)         (None, 16, 16, 64)        9472      
_________________________________________________________________
batch_normalization_88 (Batc (None, 16, 16, 64)        256       
_________________________________________________________________
activation_88 (Activation)   (None, 16, 16, 64)        0         
_________________________________________________________________
block1_maxpool (MaxPooling2D (None, 8, 8, 64)          0         
_________________________________________________________________
block2_conv1_1 (Conv2D)      (None, 8, 8, 64)          36928     
_________________________________________________________________
batch_normalization_89 (Batc (None, 8, 8, 64)          256 

In [19]:
del plaainnet_34

In [20]:
# 원하는 블록의 설계에 따라 매개변수로 리스트를 전달해 줍니다.
plaainnet_50 = build_plainnet(input_shape=(32,32,3), is_50=True)

plaainnet_50.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv (Conv2D)         (None, 16, 16, 64)        9472      
_________________________________________________________________
batch_normalization_121 (Bat (None, 16, 16, 64)        256       
_________________________________________________________________
activation_121 (Activation)  (None, 16, 16, 64)        0         
_________________________________________________________________
block1_maxpool (MaxPooling2D (None, 8, 8, 64)          0         
_________________________________________________________________
block2_conv1_1 (Conv2D)      (None, 8, 8, 64)          4160      
_________________________________________________________________
batch_normalization_122 (Bat (None, 8, 8, 64)          256 

In [21]:
del plaainnet_50

In [22]:
BATCH_SIZE = 32
EPOCH = 40
IMG_SIZE = (224, 224)

def preprocess_image(image, label):
    """이미지 크기를 일정하게 변환하는 전처리 함수"""
    image = tf.image.resize(image, IMG_SIZE)  # ✅ 이미지 크기 조정
    image = image / 255.0  # ✅ 0~1 사이로 정규화
    return image, label

In [23]:
setattr(tfds.image_classification.cats_vs_dogs, '_URL',"https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip")

ds_train, ds_info = tfds.load(
    'cats_vs_dogs',
    split=['train'],
    as_supervised=True,
    shuffle_files=True,
    with_info=True,
)
ds_train = ds_train[0]
ds_train

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/23262 [00:00<?, ? examples/s]

Shuffling cats_vs_dogs-train.tfrecord...:   0%|          | 0/23262 [00:00<?, ? examples/s]

Dataset cats_vs_dogs downloaded and prepared to /aiffel/tensorflow_datasets/cats_vs_dogs/4.0.0. Subsequent calls will reuse this data.


<_OptionsDataset shapes: ((None, None, 3), ()), types: (tf.uint8, tf.int64)>

In [24]:
# ✅ `map()`을 사용하여 데이터 변환 적용
ds_train = ds_train.map(preprocess_image)

In [25]:
num_samples = ds_info.splits['train'].num_examples
num_samples

23262

In [26]:
train_size = int(num_samples * 0.8)
test_size = num_samples - train_size  # 나머지 20%
train_size, test_size

(18609, 4653)

In [27]:
ds_train_split = ds_train.take(train_size)  # 처음 80% 가져오기
ds_test_split = ds_train.skip(train_size)   # 이후 20% 가져오기

print("Train 데이터 개수:", len(list(ds_train_split.as_numpy_iterator())))
print("Test 데이터 개수:", len(list(ds_test_split.as_numpy_iterator())))

Corrupt JPEG data: 99 extraneous bytes before marker 0xd9
Corrupt JPEG data: 396 extraneous bytes before marker 0xd9
Corrupt JPEG data: 162 extraneous bytes before marker 0xd9
Corrupt JPEG data: 252 extraneous bytes before marker 0xd9
Corrupt JPEG data: 65 extraneous bytes before marker 0xd9
Corrupt JPEG data: 1403 extraneous bytes before marker 0xd9
Corrupt JPEG data: 214 extraneous bytes before marker 0xd9


Train 데이터 개수: 18609


Corrupt JPEG data: 99 extraneous bytes before marker 0xd9
Corrupt JPEG data: 162 extraneous bytes before marker 0xd9
Corrupt JPEG data: 396 extraneous bytes before marker 0xd9
Corrupt JPEG data: 252 extraneous bytes before marker 0xd9
Corrupt JPEG data: 1403 extraneous bytes before marker 0xd9
Corrupt JPEG data: 65 extraneous bytes before marker 0xd9
Corrupt JPEG data: 214 extraneous bytes before marker 0xd9
Corrupt JPEG data: 128 extraneous bytes before marker 0xd9
Corrupt JPEG data: 2226 extraneous bytes before marker 0xd9
Corrupt JPEG data: 239 extraneous bytes before marker 0xd9
Corrupt JPEG data: 1153 extraneous bytes before marker 0xd9
Corrupt JPEG data: 228 extraneous bytes before marker 0xd9


Test 데이터 개수: 4653


In [28]:
# Train 데이터셋
ds_train = ds_train_split.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# Test 데이터셋
ds_test = ds_test_split.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# 데이터셋 확인
for image, label in ds_train.take(1):
    print("Train 이미지 크기:", image.shape)
    print("Train 레이블:", label)

Train 이미지 크기: (32, 224, 224, 3)
Train 레이블: tf.Tensor([1 0 1 1 1 0 1 0 1 0 0 0 0 1 1 0 1 0 1 1 1 1 1 1 0 1 1 0 1 0 0 0], shape=(32,), dtype=int64)


In [29]:
tf.keras.backend.clear_session()

In [ ]:
import pickle
import os

# 저장할 파일 경로
CHECKPOINT_PATH = "plaainnet_34_checkpoint.h5"
HISTORY_PATH = "plaainnet_34_history.pkl"

# 저장된 체크포인트가 있으면 로드
if os.path.exists(CHECKPOINT_PATH):
    print("🔄 기존 모델 불러오는 중...")
    model = tf.keras.models.load_model(CHECKPOINT_PATH)
else:
    print("🆕 새로운 모델 생성...")
    model = build_plainnet(input_shape=(224,224,3),
              num_cnn_list=[3,4,6,3],
              channel_list=[64,128,256,512],
              num_classes=2,
                is_50=False)

start_epoch = 0
# 저장된 학습 히스토리 불러오기
if os.path.exists(HISTORY_PATH):
    print("📂 학습 이력 로드 중...")
    with open(HISTORY_PATH, 'rb') as f:
        history_data = pickle.load(f)
    start_epoch = history_data["epoch"]
    history = history_data["history"]
else:
    print("📂 학습 이력 없음. 새로 시작합니다.")
    start_epoch = 0
    history = {"loss": [], "accuracy": []}

# 체크포인트 콜백 설정
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    CHECKPOINT_PATH, save_best_only=False, save_weights_only=False
)

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.9),
    metrics=['accuracy'],
)

# 추가 학습할 에포크 설정
history_obj = model.fit(
    ds_train,
    initial_epoch=start_epoch,  # 이어서 학습
    epochs=EPOCH,
    callbacks=[checkpoint_callback],
    use_multiprocessing=True
)

# 학습 이력 저장
history["loss"].extend(history_obj.history["loss"])
history["accuracy"].extend(history_obj.history["accuracy"])


🆕 새로운 모델 생성...
📂 학습 이력 없음. 새로 시작합니다.
Epoch 1/40
225/582 [==========>...................] - ETA: 8:36 - loss: 7.7087 - accuracy: 0.5024

Corrupt JPEG data: 99 extraneous bytes before marker 0xd9


260/582 [============>.................] - ETA: 7:49 - loss: 6.7650 - accuracy: 0.5006

270/582 [============>.................] - ETA: 7:35 - loss: 6.5402 - accuracy: 0.5006

Corrupt JPEG data: 396 extraneous bytes before marker 0xd9


...

378/582 [==================>...........] - ETA: 4:49 - loss: 0.6953 - accuracy: 0.5073

Corrupt JPEG data: 1403 extraneous bytes before marker 0xd9


539/582 [==========================>...] - ETA: 1:01 - loss: 0.6955 - accuracy: 0.5035

Corrupt JPEG data: 214 extraneous bytes before marker 0xd9


582/582 [==============================] - 827s 1s/step - loss: 0.6955 - accuracy: 0.5033
Epoch 4/40
110/582 [====>.........................] - ETA: 11:12 - loss: 0.6953 - accuracy: 0.5082

In [ ]:
with open(HISTORY_PATH, 'wb') as f:
    pickle.dump({"epoch": start_epoch + EPOCHS, "history": history}, f)

# 학습 곡선 그리기
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(history["loss"], label="Loss")
plt.legend()
plt.subplot(1,2,2)
plt.plot(history["accuracy"], label="Accuracy")
plt.legend()
plt.show()

In [ ]:


checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, monitor="val_loss", save_best_only=True, save_weights_only=True, verbose=1
)

plaainnet_34 = build_plainnet(input_shape=(224,224,3),
              num_cnn_list=[3,4,6,3],
              channel_list=[64,128,256,512],
              num_classes=2,
                is_50=False)

plaainnet_34.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.9),
    metrics=['accuracy'],
)

history_plaainnet_34 = plaainnet_34.fit(
    ds_train,
#     steps_per_epoch=int(ds_info.splits['train'].num_examples/BATCH_SIZE),
#     validation_steps=int(ds_info.splits['test'].num_examples/BATCH_SIZE),
    epochs=EPOCH,
    validation_data=ds_test,
    verbose=1,
    use_multiprocessing=True,
    callbacks=[lr_callback, checkpoint_callback]
)

Epoch 1/40
223/582 [==========>...................] - ETA: 8:22 - loss: 3.6775 - accuracy: 0.5013

Corrupt JPEG data: 99 extraneous bytes before marker 0xd9


259/582 [============>.................] - ETA: 7:31 - loss: 3.2635 - accuracy: 0.4988

271/582 [============>.................] - ETA: 7:14 - loss: 3.1498 - accuracy: 0.4985

Corrupt JPEG data: 396 extraneous bytes before marker 0xd9


...

541/582 [==========================>...] - ETA: 56s - loss: 0.6959 - accuracy: 0.5012

Corrupt JPEG data: 214 extraneous bytes before marker 0xd9


582/582 [==============================] - ETA: 0s - loss: 0.6959 - accuracy: 0.5013

Corrupt JPEG data: 99 extraneous bytes before marker 0xd9
Corrupt JPEG data: 396 extraneous bytes before marker 0xd9
Corrupt JPEG data: 162 extraneous bytes before marker 0xd9
Corrupt JPEG data: 252 extraneous bytes before marker 0xd9
Corrupt JPEG data: 65 extraneous bytes before marker 0xd9
Corrupt JPEG data: 1403 extraneous bytes before marker 0xd9
Corrupt JPEG data: 214 extraneous bytes before marker 0xd9
Corrupt JPEG data: 2226 extraneous bytes before marker 0xd9
Corrupt JPEG data: 128 extraneous bytes before marker 0xd9
Corrupt JPEG data: 239 extraneous bytes before marker 0xd9
Corrupt JPEG data: 1153 extraneous bytes before marker 0xd9
Corrupt JPEG data: 228 extraneous bytes before marker 0xd9


582/582 [==============================] - 888s 2s/step - loss: 0.6959 - accuracy: 0.5013 - val_loss: 0.6931 - val_accuracy: 0.4986

Epoch 00002: val_loss improved from 0.74043 to 0.69314, saving model to plaainnet_34_checkpoint.h5
Epoch 3/40
186/582 [========>.....................] - ETA: 9:04 - loss: 0.6952 - accuracy: 0.5077

In [ ]:
resnet_34 = build_resnet(input_shape=(224,224,3),
              num_cnn_list=[3,4,6,3],
              channel_list=[64,128,256,512],
              num_classes=2,
                is_50=False)

resnet_34.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.SGD(lr=0.01, clipnorm=1.),
    metrics=['accuracy'],
)

history_resnet_34 = resnet_34.fit(
    ds_train,
#     steps_per_epoch=int(ds_info.splits['train'].num_examples/BATCH_SIZE),
#     validation_steps=int(ds_info.splits['test'].num_examples/BATCH_SIZE),
    epochs=EPOCH,
    validation_data=ds_test,
    verbose=1,
    use_multiprocessing=True,
)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history_plaainnet_34.history['loss'], 'r')
plt.plot(history_resnet_34.history['loss'], 'b')
plt.title('Model training loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['plaainnet_34', 'resnet_34'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history_plaainnet_34.history['val_accuracy'], 'r')
plt.plot(history_resnet_34.history['val_accuracy'], 'b')
plt.title('Model validation accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['plaainnet_34', 'resnet_34'], loc='upper left')
plt.show()

In [ ]:
plaainnet_50 = build_plainnet(input_shape=(224,224,3),
              num_cnn_list=[3,4,6,3],
              channel_list=[64,128,256,512],
              num_classes=2,
                is_50=True)

plaainnet_50.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.SGD(lr=0.01, clipnorm=1.),
    metrics=['accuracy'],
)

history_plaainnet_50 = plaainnet_50.fit(
    ds_train,
#     steps_per_epoch=int(ds_info.splits['train'].num_examples/BATCH_SIZE),
#     validation_steps=int(ds_info.splits['test'].num_examples/BATCH_SIZE),
    epochs=EPOCH,
    validation_data=ds_test,
    verbose=1,
    use_multiprocessing=True,
)

In [ ]:
resnet_50 = build_resnet(input_shape=(224,224,3),
              num_cnn_list=[3,4,6,3],
              channel_list=[64,128,256,512],
              num_classes=2,
                is_50=True)

resnet_50.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.SGD(lr=0.01, clipnorm=1.),
    metrics=['accuracy'],
)

history_resnet_50 = resnet_50.fit(
    ds_train,
#     steps_per_epoch=int(ds_info.splits['train'].num_examples/BATCH_SIZE),
#     validation_steps=int(ds_info.splits['test'].num_examples/BATCH_SIZE),
    epochs=EPOCH,
    validation_data=ds_test,
    verbose=1,
    use_multiprocessing=True,
)

In [ ]:
plt.plot(history_plaainnet_50.history['loss'], 'r')
plt.plot(history_resnet_50.history['loss'], 'b')
plt.title('Model training loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['plaainnet_50', 'resnet_50'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history_plaainnet_50.history['val_accuracy'], 'r')
plt.plot(history_resnet_50.history['val_accuracy'], 'b')
plt.title('Model validation accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['plaainnet_50', 'resnet_50'], loc='upper left')
plt.show()